# ZNews HTML Category Crawler - Với Cơ chế xử lý Duplicate

Crawler này sẽ thu thập tin tức từ ZNews theo từng category bằng cách:
1. **Tải danh sách đã crawl**: Load tất cả URL và ID đã có trong CSV vào bộ nhớ
2. **Kiểm tra trước khi fetch**: Trước khi tải HTML của bài viết, kiểm tra xem URL đã tồn tại chưa
3. **Bỏ qua duplicate tự động**: Nếu URL hoặc ID đã có trong CSV, bỏ qua ngay lập tức (không tải HTML)
4. **Chỉ crawl bài mới**: Chỉ tải và xử lý HTML của những bài viết chưa có trong CSV
5. **Báo cáo thống kê**: Hiển thị số lượng bài mới, bài bị trùng, bài cũ, và tỷ lệ duplicate

## Ví dụ Output khi chạy hàng ngày:

**Lần 1 (ngày đầu):**
```
[https://znews.vn/xuat-ban.html] added=45, duplicates=0, old=5 (0.0% duplicate)
[https://znews.vn/kinh-doanh-tai-chinh.html] added=52, duplicates=0, old=3 (0.0% duplicate)
Total: 320 new, 0 duplicates, 25 old (0.0% duplicate rate)
```

**Lần 2 (ngày thứ 2):**
```
[https://znews.vn/xuat-ban.html] added=8, duplicates=40, old=7 (83.3% duplicate)
[https://znews.vn/kinh-doanh-tai-chinh.html] added=12, duplicates=45, old=5 (78.9% duplicate)
Total: 65 new, 280 duplicates, 30 old (81.2% duplicate rate)
```

**Lần 3 (ngày thứ 3):**
```
[https://znews.vn/xuat-ban.html] added=5, duplicates=42, old=8 (89.4% duplicate)
[https://znews.vn/kinh-doanh-tai-chinh.html] added=7, duplicates=50, old=6 (87.7% duplicate)
Total: 45 new, 295 duplicates, 35 old (86.8% duplicate rate)
```

## Lợi ích:
- ✅ **Không có duplicate trong CSV** - Đảm bảo mỗi URL chỉ xuất hiện 1 lần
- ✅ **Chạy nhanh** - Bỏ qua duplicate ngay trước khi fetch HTML (tiết kiệm thời gian và băng thông)
- ✅ **Tự động** - Không cần can thiệp thủ công, chỉ cần chạy script hàng ngày
- ✅ **Báo cáo rõ ràng** - Biết chính xác bao nhiêu bài mới, bao nhiêu bài trùng
- ✅ **Hiệu quả cao** - Tỷ lệ duplicate tăng dần sau mỗi lần chạy, chứng tỏ crawler đang hoạt động đúng

## Khuyến nghị:
- Chạy crawler **mỗi ngày 1 lần** để cập nhật tin tức mới nhất
- ZNews cập nhật tin trong vòng 24h, nên duplicate rate sẽ cao (80-90%) sau 2-3 ngày đầu
- Tỷ lệ duplicate cao = crawler đang hoạt động hiệu quả (không bỏ sót bài cũ, không crawl lại bài đã có)

In [ ]:
%pip install requests beautifulsoup4 lxml python-dateutil

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import csv
import os
import re
import time
import random
import hashlib
from datetime import datetime, timezone, timedelta
from turtle import pd
from typing import Optional, Dict, Any, List, Set, Tuple
from urllib.parse import urlparse, urlunparse, parse_qs, urlencode

import requests
from bs4 import BeautifulSoup
from dateutil import parser as dateparser
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

VN_TZ = timezone(timedelta(hours=7))

# ================== CONFIG ==================
CATEGORY_URLS = [
    "https://znews.vn/xuat-ban.html",
    "https://znews.vn/kinh-doanh-tai-chinh.html",
    "https://znews.vn/suc-khoe.html",
    "https://znews.vn/the-thao.html",
    "https://znews.vn/doi-song.html",
    "https://znews.vn/cong-nghe.html",
    "https://znews.vn/giai-tri.html",
]

# Crawl từ mới -> cũ cho tới khi bài có ngày < END_DATE (theo giờ VN)
END_DATE = "2026-01-15"  # YYYY-MM-DD
MAX_PAGES_PER_CATEGORY = 2000  # safety stop

CSV_PATH = "znews_html_categories_vi.csv"

TIMEOUT = 25
REQUEST_DELAY_BASE = 0.25
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ZNewsHTMLCrawler/1.0)",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}
# ===========================================

CSV_HEADER = [
    "id",
    "title",
    "published_at",
    "source.name",
    "url",
    "language",
    "category.primary",
    "keywords",
    "entities",
    "content.text",
]

SOURCE_NAME = "ZNews"
DEFAULT_LANGUAGE = "vi"
DEBUG = False

# ----- HTTP session with retry -----
session = requests.Session()
session.headers.update(HEADERS)

retry = Retry(
    total=6,
    connect=6,
    read=6,
    backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET", "HEAD"],
    respect_retry_after_header=True,
    raise_on_status=False,
)
adapter = HTTPAdapter(max_retries=retry, pool_connections=50, pool_maxsize=50)
session.mount("http://", adapter)
session.mount("https://", adapter)


def log(msg: str):
    if DEBUG:
        print(msg)


def polite_sleep():
    time.sleep(REQUEST_DELAY_BASE + random.uniform(0, 0.4))


def md5_id(text: str) -> str:
    return hashlib.md5(text.encode("utf-8")).hexdigest()


def fetch_text(url: str) -> str:
    r = session.get(url, timeout=TIMEOUT)
    r.raise_for_status()
    return r.text


def to_iso_utc(s: Optional[str]) -> Optional[str]:
    if not s:
        return None
    try:
        dt = dateparser.parse(s)
        if not dt:
            return None
        if dt.tzinfo is None:
            if VN_TZ:
                dt = dt.replace(tzinfo=VN_TZ)
            else:
                dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(timezone.utc).isoformat()
    except Exception:
        return None


def iso_to_local_date(iso_utc: str) -> Optional[str]:
    if not iso_utc:
        return None
    try:
        dt = dateparser.parse(iso_utc)
        if not dt:
            return None
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        if VN_TZ:
            dt_local = dt.astimezone(VN_TZ)
        else:
            dt_local = dt
        return dt_local.date().isoformat()
    except Exception:
        return None


def ensure_csv_header(csv_path: str):
    if not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0:
        with open(csv_path, "w", encoding="utf-8", newline="") as f:
            csv.writer(f).writerow(CSV_HEADER)


def load_seen_from_csv(csv_path: str) -> Tuple[Set[str], Set[str]]:
    seen_urls, seen_ids = set(), set()
    if not os.path.exists(csv_path):
        return seen_urls, seen_ids
    try:
        with open(csv_path, "r", encoding="utf-8", newline="") as f:
            r = csv.reader(f)
            header = next(r, None)
            if not header:
                return seen_urls, seen_ids
            id_idx = header.index("id") if "id" in header else 0
            url_idx = header.index("url") if "url" in header else 4
            for row in r:
                if len(row) > url_idx:
                    u = row[url_idx].strip()
                    if u:
                        seen_urls.add(u)
                if len(row) > id_idx:
                    i = row[id_idx].strip()
                    if i:
                        seen_ids.add(i)
    except Exception:
        pass
    return seen_urls, seen_ids


def append_row(csv_path: str, row: Dict[str, Any]):
    with open(csv_path, "a", encoding="utf-8", newline="") as f:
        w = csv.writer(f)
        w.writerow([row.get(k, "") for k in CSV_HEADER])
        f.flush()


def extract_language_from_html(soup: BeautifulSoup) -> str:
    html_tag = soup.find("html")
    if html_tag:
        lang = html_tag.get("lang") or html_tag.get("xml:lang")
        if lang:
            lang = lang.lower().strip()
            if lang.startswith("vi"):
                return "vi"
            if lang.startswith("en"):
                return "en"
            return lang
    return DEFAULT_LANGUAGE


def extract_keywords_from_html(soup: BeautifulSoup) -> List[str]:
    for sel in ['meta[name="keywords"]', 'meta[name="news_keywords"]']:
        tag = soup.select_one(sel)
        if tag and tag.get("content"):
            raw = tag["content"]
            kws = [x.strip() for x in raw.split(",") if x.strip()]
            seen = set()
            out = []
            for k in kws:
                if k not in seen:
                    seen.add(k)
                    out.append(k)
            return out
    return []


def extract_article_meta(article_html: str) -> Dict[str, Any]:
    soup = BeautifulSoup(article_html, "lxml")

    # title
    title = ""
    og = soup.select_one('meta[property="og:title"]')
    if og and og.get("content"):
        title = og["content"].strip()
    if not title:
        h1 = soup.select_one("h1.title-detail, h1.article-title, h1")
        if h1:
            title = h1.get_text(strip=True)

    # published_at
    pub = ""
    m_pub = soup.select_one('meta[property="article:published_time"]')
    if m_pub and m_pub.get("content"):
        pub = to_iso_utc(m_pub["content"].strip()) or ""
    if not pub:
        m2 = soup.select_one('meta[itemprop="datePublished"]')
        if m2 and m2.get("content"):
            pub = to_iso_utc(m2["content"].strip()) or ""
    if not pub:
        ttag = soup.select_one("time")
        if ttag:
            pub = to_iso_utc(ttag.get("datetime") or ttag.get_text(strip=True)) or ""

    # category.primary
    category_primary = ""
    sec = soup.select_one('meta[property="article:section"]')
    if sec and sec.get("content"):
        category_primary = sec["content"].strip()

    language = extract_language_from_html(soup)
    keywords = extract_keywords_from_html(soup)

    # content.text - ZNews thường dùng class .article-body, .the-article-body
    content_text = ""
    article_body = soup.select_one(".the-article-body")
    if not article_body:
        article_body = soup.select_one(".article-body")
    if not article_body:
        article_body = soup.select_one("article")
    if not article_body:
        article_body = soup.select_one(".content-detail")
    
    if article_body:
        paragraphs = article_body.find_all("p")
        text_parts = []
        for p in paragraphs:
            text = p.get_text(strip=True)
            if text:
                text_parts.append(text)
        content_text = " ".join(text_parts)

    return {
        "title": title,
        "published_at": pub,
        "language": language,
        "keywords": keywords,
        "category_from_article": category_primary,
        "entities": [],
        "content_text": content_text,
    }


def extract_article_urls_from_category_page(html: str) -> List[str]:
    soup = BeautifulSoup(html, "lxml")

    urls = []
    for a in soup.select("a[href]"):
        href = a.get("href", "").strip()
        if not href:
            continue
        if href.startswith("/"):
            href = "https://znews.vn" + href
        if not href.startswith("https://znews.vn/"):
            continue
        # ZNews bài viết thường có format /ten-bai-post[ID].html
        if "-post" in href and ".html" in href:
            urls.append(href.split("?")[0])

    # unique giữ thứ tự
    seen = set()
    out = []
    for u in urls:
        if u not in seen:
            seen.add(u)
            out.append(u)
    return out


def find_next_page_url(category_url: str, html: str, current_page: int) -> Optional[str]:
    soup = BeautifulSoup(html, "lxml")

    # thử rel=next
    ln = soup.select_one('link[rel="next"]')
    if ln and ln.get("href"):
        href = ln["href"].strip()
        if href.startswith("/"):
            href = "https://znews.vn" + href
        return href

    # thử tìm nút pagination
    a_next = soup.select_one('a.page-next, a[rel="next"]')
    if a_next and a_next.get("href"):
        href = a_next["href"].strip()
        if href.startswith("/"):
            href = "https://znews.vn" + href
        return href

    # fallback: ZNews dùng format /trangX.html
    # https://znews.vn/the-thao.html -> https://znews.vn/the-thao/trang2.html
    next_page = current_page + 1
    base_url = category_url.rstrip("/").replace(".html", "")
    return f"{base_url}/trang{next_page}.html"


def make_row(url: str, meta: Dict[str, Any], category_fallback: str) -> Dict[str, Any]:
    id_ = md5_id(url)
    category_primary = meta.get("category_from_article") or category_fallback
    keywords_str = "|".join(meta.get("keywords") or [])
    entities_str = "|".join(meta.get("entities") or [])

    return {
        "id": id_,
        "title": meta.get("title") or "",
        "published_at": meta.get("published_at") or "",
        "source.name": SOURCE_NAME,
        "url": url,
        "language": meta.get("language") or DEFAULT_LANGUAGE,
        "category.primary": category_primary or "",
        "keywords": keywords_str,
        "entities": entities_str,
        "content.text": meta.get("content_text") or "",
    }


def crawl_category(category_url: str, end_date: str, seen_urls: Set[str], seen_ids: Set[str]) -> Tuple[int, int, int]:
    """
    Crawl category và trả về (added, skipped_duplicate, skipped_old)
    - added: số bài mới được thêm vào CSV
    - skipped_duplicate: số bài bị trùng (đã có trong CSV)
    - skipped_old: số bài cũ hơn END_DATE
    """
    added = 0
    skipped_duplicate = 0
    skipped_old = 0
    page = 1
    url_page = category_url

    # extract category slug từ URL
    category_slug = category_url.rstrip("/").split("/")[-1].replace(".html", "")

    while page <= MAX_PAGES_PER_CATEGORY and url_page:
        html = fetch_text(url_page)
        article_urls = extract_article_urls_from_category_page(html)

        if DEBUG:
            log(f"[{category_slug}] page {page} got {len(article_urls)} candidate urls: {url_page}")

        if not article_urls:
            break

        page_all_older_than_end = True

        for aurl in article_urls:
            # Kiểm tra duplicate TRƯỚC KHI fetch HTML
            if aurl in seen_urls:
                skipped_duplicate += 1
                continue

            aid = md5_id(aurl)
            if aid in seen_ids:
                skipped_duplicate += 1
                continue

            try:
                ah = fetch_text(aurl)
                meta = extract_article_meta(ah)
            except Exception as e:
                log(f"[WARN] article fetch failed {aurl}: {e}")
                continue
            finally:
                polite_sleep()

            pub_iso = meta.get("published_at") or ""
            pub_local_date = iso_to_local_date(pub_iso) or ""

            # nếu có ngày và nhỏ hơn end_date => đánh dấu cũ
            if pub_local_date and pub_local_date < end_date:
                skipped_old += 1
                pass
            else:
                page_all_older_than_end = False

            # Nếu bài >= end_date thì ghi
            if (not pub_local_date) or (pub_local_date >= end_date):
                row = make_row(aurl, meta, category_fallback=category_slug)
                append_row(CSV_PATH, row)
                seen_urls.add(aurl)
                seen_ids.add(aid)
                added += 1

        # Nếu cả trang toàn bài cũ hơn end_date thì dừng category này
        if page_all_older_than_end:
            if DEBUG:
                log(f"[{category_slug}] stop: page {page} all older than end_date={end_date}")
            break

        # đi trang tiếp
        next_url = find_next_page_url(category_url, html, current_page=page)
        if not next_url or next_url == url_page:
            break
        if next_url == url_page:
            break
        url_page = next_url
        page += 1
        polite_sleep()

    return added, skipped_duplicate, skipped_old


def main():
    ensure_csv_header(CSV_PATH)
    seen_urls, seen_ids = load_seen_from_csv(CSV_PATH)

    print(f"=== ZNews Crawler - Cơ chế xử lý Duplicate ===")
    print(f"Đã load {len(seen_urls)} URLs và {len(seen_ids)} IDs từ CSV")
    print(f"END_DATE: {END_DATE}")
    print(f"Crawling {len(CATEGORY_URLS)} categories...\n")

    total_added = 0
    total_duplicates = 0
    total_old = 0
    
    for cat in CATEGORY_URLS:
        try:
            added, skipped_duplicate, skipped_old = crawl_category(cat, END_DATE, seen_urls, seen_ids)
            total_added += added
            total_duplicates += skipped_duplicate
            total_old += skipped_old
            
            # Tính tỷ lệ duplicate
            total_found = added + skipped_duplicate + skipped_old
            dup_rate = (skipped_duplicate / total_found * 100) if total_found > 0 else 0.0
            
            print(f"[{cat}]")
            print(f"  ✅ Added: {added} bài mới")
            print(f"  🔄 Duplicates: {skipped_duplicate} bài trùng")
            print(f"  ⏰ Old: {skipped_old} bài cũ (< {END_DATE})")
            print(f"  📊 Duplicate rate: {dup_rate:.1f}%\n")
            
        except Exception as e:
            print(f"[{cat}] ❌ ERROR: {e}\n")

    # Tổng kết
    grand_total = total_added + total_duplicates + total_old
    overall_dup_rate = (total_duplicates / grand_total * 100) if grand_total > 0 else 0.0
    import pandas as pd

    df = pd.read_csv("znews_html_categories_vi.csv")
    
    print(f"\n{'='*60}")
    print(f"📈 TỔNG KẾT:")
    print(f"  ✅ Tổng bài: {len(df)}")
    print(f"  ✅ Tổng bài mới thêm vào CSV: {total_added}")
    print(f"  🔄 Tổng bài trùng (bỏ qua): {total_duplicates}")
    print(f"  ⏰ Tổng bài cũ (bỏ qua): {total_old}")
    print(f"  📊 Tổng bài kiểm tra: {grand_total}")
    print(f"  💯 Tỷ lệ duplicate: {overall_dup_rate:.1f}%")
    print(f"{'='*60}")
    print(f"\n✅ Hoàn thành! Đã thêm {total_added} bài mới vào {CSV_PATH}")
    
    if overall_dup_rate > 70:
        print(f"💡 Gợi ý: Tỷ lệ duplicate cao ({overall_dup_rate:.1f}%) cho thấy crawler đang hoạt động tốt!")
    elif overall_dup_rate < 20 and total_duplicates > 0:
        print(f"⚠️  Lưu ý: Tỷ lệ duplicate thấp ({overall_dup_rate:.1f}%) - có thể có nhiều bài mới hoặc nguồn cập nhật thường xuyên")


if __name__ == "__main__":
    main()

=== ZNews Crawler - Cơ chế xử lý Duplicate ===
Đã load 2851 URLs và 2851 IDs từ CSV
END_DATE: 2026-01-15
Crawling 7 categories...

[https://znews.vn/xuat-ban.html]
  ✅ Added: 13 bài mới
  🔄 Duplicates: 87 bài trùng
  ⏰ Old: 12 bài cũ (< 2026-01-15)
  📊 Duplicate rate: 77.7%

[https://znews.vn/kinh-doanh-tai-chinh.html]
  ✅ Added: 35 bài mới
  🔄 Duplicates: 75 bài trùng
  ⏰ Old: 12 bài cũ (< 2026-01-15)
  📊 Duplicate rate: 61.5%

[https://znews.vn/suc-khoe.html]
  ✅ Added: 16 bài mới
  🔄 Duplicates: 98 bài trùng
  ⏰ Old: 28 bài cũ (< 2026-01-15)
  📊 Duplicate rate: 69.0%

[https://znews.vn/the-thao.html]
  ✅ Added: 49 bài mới
  🔄 Duplicates: 54 bài trùng
  ⏰ Old: 6 bài cũ (< 2026-01-15)
  📊 Duplicate rate: 49.5%

[https://znews.vn/doi-song.html]
  ✅ Added: 20 bài mới
  🔄 Duplicates: 81 bài trùng
  ⏰ Old: 30 bài cũ (< 2026-01-15)
  📊 Duplicate rate: 61.8%

[https://znews.vn/cong-nghe.html]
  ✅ Added: 7 bài mới
  🔄 Duplicates: 93 bài trùng
  ⏰ Old: 6 bài cũ (< 2026-01-15)
  📊 Duplicate ra

In [ ]:
import pandas as pd
df = pd.read_csv("znews_html_categories_vi.csv")
print(f"  ✅ Tổng bài: {len(df)}")